# Running DeepSeek-R1 on Google Colab

This notebook will guide you through the steps to install Ollama, download DeepSeek-R1 models, and run them on Google Colab.

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# Download DeepSeek-R1 model
!ollama pull deepseek-r1:7b

In [ ]:
# Run DeepSeek-R1 model
!ollama run deepseek-r1:7b